en esta version cambio el modelo para evitar sobreajuste, incorporo a los clusters y a la red 4 variables nuevas: dif entre dic y feb, totales 2019 y 2018

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMinMax
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
import joblib
import warnings
import tensorflow as tf
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
warnings.filterwarnings("ignore", category=UserWarning, module='keras')
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Reshape, Bidirectional, LSTM, Dense, Dropout, Activation
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping  # Importa EarlyStopping desde callbacks
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
import tensorflow as tf
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.clustering import TimeSeriesKMeans
import joblib

In [30]:
# Cargar y preprocesar los datos
file_path= "C:/Users/vgarciario/desktop/UA MASTER/labo3/scripts/labo3-2024v/final_dataset_completo_con_ceros.csv"
#file_path = "C:/Users/Usuario/desktop/vero2/final_dataset_completo_con_ceros.csv"
df = pd.read_csv(file_path, sep='\t')

df = df.drop('Unnamed: 0', axis=1)
df.head(12)

# filtered_df = df[df['product_id'].between(20001, 20012)]

# Opcional: Ver las primeras filas del DataFrame filtrado
# print(filtered_df.head())

# df=filtered_df

,customer_id,product_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,descripcion,quarter,month,close_quarter,age,mes_inicial
0,10001,20001,201812,0.0,20.0,254.62373,254.62373,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q4,12,1.0,23.0,2018-12-01
1,10001,20001,201901,0.0,53.0,393.26092,386.60688,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,1,0.0,24.0,2018-12-01
2,10001,20001,201902,0.0,39.0,309.90610,309.90610,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,2,0.0,25.0,2018-12-01
3,10001,20001,201903,0.0,23.0,142.87158,130.54927,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,3,1.0,26.0,2018-12-01
4,10001,20001,201904,0.0,33.0,364.37071,364.37071,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q2,4,0.0,27.0,2018-12-01
5,10001,20001,201905,0.0,31.0,439.90647,439.90647,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q2,5,0.0,28.0,2018-12-01
6,10001,20001,201906,0.0,7.0,65.92436,65.92436,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q2,6,1.0,29.0,2018-12-01
7,10001,20001,201907,0.0,14.0,144.78714,144.78714,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q3,7,0.0,30.0,2018-12-01
8,10001,20001,201908,0.0,9.0,33.63991,33.63991,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q3,8,0.0,31.0,2018-12-01
9,10001,20001,201909,0.0,18.0,111.51691,109.05244,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q3,9,1.0,32.0,2018-12-01


In [121]:
df.columns

Index(['customer_id', 'product_id', 'periodo', 'plan_precios_cuidados',
       'cust_request_qty', 'cust_request_tn', 'tn', 'cat1', 'cat2', 'cat3',
       'brand', 'sku_size', 'descripcion', 'quarter', 'month', 'close_quarter',
       'age', 'mes_inicial'],
      dtype='object')

In [31]:
# Reemplazar 082019 por promedio 07 y 09
df['periodo'] = df['periodo'].astype(str).str.strip()
df_filtered = df[df['periodo'].isin(['201907', '201908', '201909'])]
pivoted_sales = df_filtered.pivot_table(index=['product_id', 'customer_id'], columns='periodo', values='tn').reset_index()
pivoted_sales = pivoted_sales.reindex(columns=['product_id', 'customer_id', '201907', '201908', '201909'])
pivoted_sales['201908'] = pivoted_sales[['201907', '201909']].mean(axis=1)
updated_sales = pivoted_sales.melt(id_vars=['product_id', 'customer_id'], value_vars=['201907', '201908', '201909'], var_name='periodo', value_name='tn')
df.set_index(['product_id', 'customer_id', 'periodo'], inplace=True)
df.update(updated_sales.set_index(['product_id', 'customer_id', 'periodo']))
df.reset_index(inplace=True)

# Aplicar LabelEncoder a las columnas categóricas
categorical_cols = ['cat1', 'cat2', 'cat3', 'brand', 'descripcion']
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Agrupar las ventas por periodo, cat1, cat2, cat3, brand y customer_id
df['periodo'] = pd.to_datetime(df['periodo'], format='%Y%m', errors='coerce')
grouped_df = df.groupby(['periodo', 'cat1', 'cat2', 'cat3', 'brand', 'customer_id']).agg({'tn': 'sum'}).reset_index()


# Paso 2: Calcular los ratios incluyendo customer_id
df_diciembre_2019 = df[(df['periodo'].dt.year == 2019) & (df['periodo'].dt.month == 12)]
grouped_sales_2019 = df_diciembre_2019.groupby(['cat1', 'cat2', 'cat3', 'brand', 'customer_id', 'product_id'])['tn'].sum().reset_index()
group_totals_2019 = df_diciembre_2019.groupby(['cat1', 'cat2', 'cat3', 'brand', 'customer_id'])['tn'].sum().reset_index()
ratios_2019 = pd.merge(grouped_sales_2019, group_totals_2019, on=['cat1', 'cat2', 'cat3', 'brand', 'customer_id'], suffixes=('', '_total'))
ratios_2019['ratio'] = ratios_2019['tn'] / ratios_2019['tn_total']
ratio_dict = ratios_2019.set_index(['cat1', 'cat2', 'cat3', 'brand', 'customer_id', 'product_id'])['ratio'].to_dict()

In [32]:
grouped_df.head()

,periodo,cat1,cat2,cat3,brand,customer_id,tn
0,2018-12-01,0,0,4,22,10001,0.87535
1,2018-12-01,0,0,4,22,10002,0.27780
2,2018-12-01,0,0,4,22,10003,0.27256
3,2018-12-01,0,0,4,22,10004,0.13628
4,2018-12-01,0,0,4,22,10005,0.06290


In [33]:
grouped_df.columns

Index(['periodo', 'cat1', 'cat2', 'cat3', 'brand', 'customer_id', 'tn'], dtype='object')

In [34]:
# Supongamos que `grouped_df` es tu DataFrame
# Asegúrate de que la columna 'periodo' esté en formato de fecha
grouped_df['periodo'] = pd.to_datetime(grouped_df['periodo'])

# Filtrar los datos necesarios para los cálculos
df_dic2017 = grouped_df[grouped_df['periodo'] == '2017-12-01']
df_feb2018 = grouped_df[grouped_df['periodo'] == '2018-02-01']
df_dic2018 = grouped_df[grouped_df['periodo'] == '2018-12-01']
df_feb2019 = grouped_df[grouped_df['periodo'] == '2019-02-01']
df_2018 = grouped_df[grouped_df['periodo'].dt.year == 2018]
df_2019 = grouped_df[grouped_df['periodo'].dt.year == 2019]

# Merge para obtener las variaciones
merged_2018 = pd.merge(df_feb2018, df_dic2017, on=['cat1', 'cat2', 'cat3', 'brand', 'customer_id'], suffixes=('_feb2018', '_dic2017'))
merged_2019 = pd.merge(df_feb2019, df_dic2018, on=['cat1', 'cat2', 'cat3', 'brand', 'customer_id'], suffixes=('_feb2019', '_dic2018'))

# Calcular las variaciones
merged_2018['variacion_feb2018_vs_dic2017'] = merged_2018['tn_feb2018'] - merged_2018['tn_dic2017']
merged_2019['variacion_feb2019_vs_dic2018'] = merged_2019['tn_feb2019'] - merged_2019['tn_dic2018']

# Agrupar y sumar para obtener las sumas totales de tn para 2018 y 2019
sum_2018 = df_2018.groupby(['cat1', 'cat2', 'cat3', 'brand', 'customer_id'])['tn'].sum().reset_index().rename(columns={'tn': 'suma_total_2018'})
sum_2019 = df_2019.groupby(['cat1', 'cat2', 'cat3', 'brand', 'customer_id'])['tn'].sum().reset_index().rename(columns={'tn': 'suma_total_2019'})

# Unir todos los resultados en un solo DataFrame
final_df = merged_2018[['cat1', 'cat2', 'cat3', 'brand', 'customer_id', 'variacion_feb2018_vs_dic2017']].merge(
    merged_2019[['cat1', 'cat2', 'cat3', 'brand', 'customer_id', 'variacion_feb2019_vs_dic2018']],
    on=['cat1', 'cat2', 'cat3', 'brand', 'customer_id'], how='outer'
).merge(
    sum_2018, on=['cat1', 'cat2', 'cat3', 'brand', 'customer_id'], how='outer'
).merge(
    sum_2019, on=['cat1', 'cat2', 'cat3', 'brand', 'customer_id'], how='outer'
)

# Agregar las columnas 'tn' y 'periodo' originales
final_df = final_df.merge(grouped_df[['cat1', 'cat2', 'cat3', 'brand', 'customer_id', 'tn', 'periodo']], on=['cat1', 'cat2', 'cat3', 'brand', 'customer_id'], how='left')

# Rellenar NaNs con 0
final_df.fillna(0, inplace=True)

In [35]:
final_df.head()

,cat1,cat2,cat3,brand,customer_id,variacion_feb2018_vs_dic2017,variacion_feb2019_vs_dic2018,suma_total_2018,suma_total_2019,tn,periodo
0,0,0,4,22,10001,0.0,-0.60803,0.87535,3.28124,0.87535,2018-12-01
1,0,0,4,22,10001,0.0,-0.60803,0.87535,3.28124,0.30925,2019-01-01
2,0,0,4,22,10001,0.0,-0.60803,0.87535,3.28124,0.26732,2019-02-01
3,0,0,4,22,10001,0.0,-0.60803,0.87535,3.28124,0.38264,2019-03-01
4,0,0,4,22,10001,0.0,-0.60803,0.87535,3.28124,0.06290,2019-04-01


In [36]:
#final_df tiene datos originales, final_df_es esta escalado
final_df_esc=final_df

In [37]:
# Columnas a escalar
columns_to_scale = [
    'variacion_feb2018_vs_dic2017',
    'variacion_feb2019_vs_dic2018',
    'suma_total_2018',
    'suma_total_2019',
    'tn'
]

# Inicializar el StandardScaler
scaler = StandardScaler()

# Ajustar y transformar las columnas seleccionadas
final_df_esc[columns_to_scale] = scaler.fit_transform(final_df_esc[columns_to_scale])

final_df_esc.head()

,cat1,cat2,cat3,brand,customer_id,variacion_feb2018_vs_dic2017,variacion_feb2019_vs_dic2018,suma_total_2018,suma_total_2019,tn,periodo
0,0,0,4,22,10001,0.0,-0.136657,0.023427,-0.093435,-0.003176,2018-12-01
1,0,0,4,22,10001,0.0,-0.136657,0.023427,-0.093435,-0.078741,2019-01-01
2,0,0,4,22,10001,0.0,-0.136657,0.023427,-0.093435,-0.084338,2019-02-01
3,0,0,4,22,10001,0.0,-0.136657,0.023427,-0.093435,-0.068945,2019-03-01
4,0,0,4,22,10001,0.0,-0.136657,0.023427,-0.093435,-0.111625,2019-04-01


In [43]:
# Pivotear el DataFrame
pivoted_df = final_df_esc.pivot_table(index=['cat1', 'cat2', 'cat3', 'brand', 'customer_id', 'variacion_feb2018_vs_dic2017', 'variacion_feb2019_vs_dic2018', 'suma_total_2018', 'suma_total_2019'], columns='periodo', values='tn').fillna(0)
pivoted_df.head()

periodo                                                                                                                     2018-12-01  \
cat1 cat2 cat3 brand customer_id variacion_feb2018_vs_dic2017 variacion_feb2019_vs_dic2018 suma_total_2018 suma_total_2019               
0    0    4    22    10001       0.0                          -0.136657                     0.023427       -0.093435         -0.003176   
                     10002       0.0                          -0.063241                    -0.066108       -0.118003         -0.082939   
                     10003       0.0                          -0.062076                    -0.066893       -0.102973         -0.083639   
                     10004       0.0                           0.089413                    -0.087313       -0.066632         -0.101830   
                     10005       0.0                           0.014832                    -0.098308       -0.124383         -0.111625   

periodo                                                                                                                     2019-01-01  \
cat1 cat2 cat3 brand customer_id variacion_feb2018_vs_dic2017 variacion_feb2019_vs_dic2018 suma_total_2018 suma_total_2019               
0    0    4    22    10001       0.0                          -0.136657                     0.023427       -0.093435         -0.078741   
                     10002       0.0                          -0.063241                    -0.066108       -0.118003         -0.110926   
                     10003       0.0                          -0.062076                    -0.066893       -0.102973         -0.092734   
                     10004       0.0                           0.089413                    -0.087313       -0.066632         -0.083639   
                     10005       0.0                           0.014832                    -0.098308       -0.124383         -0.113725   

periodo                                                                                                                     2019-02-01  \
cat1 cat2 cat3 brand customer_id variacion_feb2018_vs_dic2017 variacion_feb2019_vs_dic2018 suma_total_2018 suma_total_2019               
0    0    4    22    10001       0.0                          -0.136657                     0.023427       -0.093435         -0.084338   
                     10002       0.0                          -0.063241                    -0.066108       -0.118003         -0.120021   
                     10003       0.0                          -0.062076                    -0.066893       -0.102973         -0.120021   
                     10004       0.0                           0.089413                    -0.087313       -0.066632         -0.047255   
                     10005       0.0                           0.014832                    -0.098308       -0.124383         -0.101830   

periodo                                                                                                                     2019-03-01  \
cat1 cat2 cat3 brand customer_id variacion_feb2018_vs_dic2017 variacion_feb2019_vs_dic2018 suma_total_2018 suma_total_2019               
0    0    4    22    10001       0.0                          -0.136657                     0.023427       -0.093435         -0.068945   
                     10002       0.0                          -0.063241                    -0.066108       -0.118003         -0.120021   
                     10003       0.0                          -0.062076                    -0.066893       -0.102973         -0.065447   
                     10004       0.0                           0.089413                    -0.087313       -0.066632         -0.038159   
                     10005       0.0                           0.014832                    -0.098308       -0.124383         -0.113024   

periodo                                                                                                                     2019-04-

In [44]:
# Convertir el DataFrame pivoteado a un array numpy
X = pivoted_df.values

# Asegurar que X tiene la forma (n_samples, n_timesteps, n_features)
X = np.expand_dims(X, axis=-1)  # Agregar una dimensión para n_features si es necesario

# Comprobar la forma de X
print(X.shape)  # Debería ser (n_samples, n_timesteps, n_features)


(40383, 13, 1)


In [45]:
# Crear el modelo TimeSeriesKMeans
n_clusters = 15
model = TimeSeriesKMeans(n_clusters=n_clusters, metric="dtw", random_state=0)

# Ajustar el modelo usando las series temporales escaladas
model.fit(X)

# Obtener los clusters asignados a cada serie temporal
clusters = model.labels_

# Agregar los clusters al DataFrame pivoteado
X['cluster'] = clusters

# Reiniciar el índice para que `pivoted_df` tenga los índices como columnas normales
X.reset_index(inplace=True)

# Fusionar `final_df` con `pivoted_df` usando `left_on` y `right_on`
grouped_dff = final_df.merge(X[['cat1', 'cat2', 'cat3', 'brand', 'customer_id', 'cluster']], 
                             on=['cat1', 'cat2', 'cat3', 'brand', 'customer_id'], 
                             how='left')

# Mostrar el DataFrame final con los números de grupo
display(grouped_dff)


KeyboardInterrupt: 

In [47]:
grouped_dff.to_csv("C:/Users/Usuario/desktop/vero2/grouped_dff.csv", index=False)


In [49]:
#control
# Dropear la columna 'periodo'
check = grouped_dff.drop(columns=['periodo'])

# Agrupar por 'cat1', 'cat2', 'cat3', 'brand', 'customer_id' y obtener el cluster asignado
result_df = check.groupby(['cat1', 'cat2', 'cat3', 'brand', 'customer_id','variacion_feb2018_vs_dic2017','variacion_feb2019_vs_dic2018','suma_total_2018','suma_total_2019']).agg({
    'cluster': 'first'
}).reset_index()

print(result_df)


       cat1  cat2  cat3  brand  customer_id  variacion_feb2018_vs_dic2017  \
0         0     0     4     22        10001                           0.0   
1         0     0     4     22        10002                           0.0   
2         0     0     4     22        10003                           0.0   
3         0     0     4     22        10004                           0.0   
4         0     0     4     22        10005                           0.0   
...     ...   ...   ...    ...          ...                           ...   
40378     3    13    82     32        10363                           0.0   
40379     3    13    82     32        10367                           0.0   
40380     3    13    82     32        10482                           0.0   
40381     3    13    82     32        10513                           0.0   
40382     3    13    82     32        10552                           0.0   

       variacion_feb2019_vs_dic2018  suma_total_2018  suma_total_2019  clus

In [50]:
# Agrupar por 'cat1', 'cat2', 'cat3', 'brand', 'customer_id' y contar clusters únicos
cluster_counts = check.groupby(['cat1', 'cat2', 'cat3', 'brand', 'customer_id','variacion_feb2018_vs_dic2017','variacion_feb2019_vs_dic2018','suma_total_2018','suma_total_2019']).agg({
    'cluster': 'nunique'
}).reset_index()

# Filtrar combinaciones con más de un cluster
multiple_clusters = cluster_counts[cluster_counts['cluster'] > 1]

# Mostrar combinaciones con más de un cluster
print(multiple_clusters)


Empty DataFrame
Columns: [cat1, cat2, cat3, brand, customer_id, variacion_feb2018_vs_dic2017, variacion_feb2019_vs_dic2018, suma_total_2018, suma_total_2019, cluster]
Index: []


hasta aca ok con grouped_dff guardado en vero2

In [51]:
# Lista para acumular las filas de resultados
resultados_por_producto = []

# Elegir un número de pasos de tiempo
n_steps = 13  # Ventana de tiempo de 18 meses
n_features = 10 # Cambia esto si tienes más características
step_ahead= 2

def crear_secuencias(datos, n_steps, step_ahead=1):
    X, y = [], []
    for i in range(len(datos) - n_steps - step_ahead + 1):
        end_ix = i + n_steps
        out_end_ix = end_ix + step_ahead - 1
        seq_x, seq_y = datos[i:end_ix], datos[out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)



from tensorflow.keras.regularizers import l2

def build_model(input_shape):
    model = Sequential()
    model.add(Bidirectional(LSTM(units=256, return_sequences=True, kernel_regularizer=l2(0.01)), input_shape=(input_shape[0], input_shape[1])))
    model.add(Dropout(0.5))  # Increased dropout rate
    model.add(LSTM(units=128, kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.5))  # Increased dropout rate
    model.add(Dense(64, kernel_regularizer=l2(0.01)))
    model.add(Activation('tanh'))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='mse')
    return model

# Use the updated early_stopping
early_stopping = EarlyStopping(monitor='loss', patience=30, restore_best_weights=True)


In [53]:
# Reescalar 'tn' en grouped_df
scaler = StandardScaler()
grouped_dff['tn_scaled'] = scaler.fit_transform(grouped_dff[['tn']])
grouped_dff['suma_total_2019_scaled'] = scaler.fit_transform(grouped_dff[['suma_total_2019']])
grouped_dff['suma_total_2018_scaled'] = scaler.fit_transform(grouped_dff[['suma_total_2018']])


ahora tengo grouped_dff con escalado en tn: se agrega columna tn_scaled

In [54]:
grouped_dff.head()

,cat1,cat2,cat3,brand,customer_id,variacion_feb2018_vs_dic2017,variacion_feb2019_vs_dic2018,suma_total_2018,suma_total_2019,tn,periodo,cluster,tn_scaled,suma_total_2019_scaled,suma_total_2018_scaled
0,0,0,4,22,10001,0.0,-0.60803,0.87535,3.28124,0.87535,2018-12-01,0,-0.003176,-0.093435,0.023427
1,0,0,4,22,10001,0.0,-0.60803,0.87535,3.28124,0.30925,2019-01-01,0,-0.078741,-0.093435,0.023427
2,0,0,4,22,10001,0.0,-0.60803,0.87535,3.28124,0.26732,2019-02-01,0,-0.084338,-0.093435,0.023427
3,0,0,4,22,10001,0.0,-0.60803,0.87535,3.28124,0.38264,2019-03-01,0,-0.068945,-0.093435,0.023427
4,0,0,4,22,10001,0.0,-0.60803,0.87535,3.28124,0.06290,2019-04-01,0,-0.111625,-0.093435,0.023427


In [ ]:
# ver de dejar solo la fecha y tn y cluster

In [56]:
# Seleccionar solo las columnas 'periodo', 'tn' y 'cluster'
time_series_dff = grouped_dff[['periodo', 'tn_scaled', 'suma_total_2019_scaled','suma_total_2018_scaled','cluster','variacion_feb2018_vs_dic2017','variacion_feb2019_vs_dic2018']]

# Mostrar el resultado
print(time_series_dff)


          periodo  tn_scaled  suma_total_2019_scaled  suma_total_2018_scaled  \
0      2018-12-01  -0.003176               -0.093435                0.023427   
1      2019-01-01  -0.078741               -0.093435                0.023427   
2      2019-02-01  -0.084338               -0.093435                0.023427   
3      2019-03-01  -0.068945               -0.093435                0.023427   
4      2019-04-01  -0.111625               -0.093435                0.023427   
...           ...        ...                     ...                     ...   
434443 2019-08-01  -0.120021               -0.134593               -0.107733   
434444 2019-09-01  -0.120021               -0.134593               -0.107733   
434445 2019-10-01  -0.120021               -0.134593               -0.107733   
434446 2019-11-01  -0.120021               -0.134593               -0.107733   
434447 2019-12-01  -0.120021               -0.134593               -0.107733   

        cluster  variacion_feb2018_vs_d

OJO DEJE EPOCHS EN 100 para version 2001 paciencia 20, 
version 20011: 200 epochs, paciencia 30

In [ ]:
# Imprimir dimensiones de X_padded y y antes de entrenar
print(f'Dimensiones de X_padded: {X_padded.shape}')
print(f'Dimensiones de y: {y.shape}')


In [57]:
from keras.callbacks import ReduceLROnPlateau
import numpy as np
import pandas as pd

models = {}
predictions = []

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=0.00001, verbose=1)

# Preparar los datos por cluster
for cluster in range(n_clusters):
    print(f'Entrenando cluster numero: {cluster}')
    cluster_data = grouped_dff[grouped_dff['cluster'] == cluster].copy()
    cluster_data.sort_values(by='periodo', inplace=True)
    
    # Preparar los datos para LSTM
    X, y = [], []
    for key, data in cluster_data.groupby(['cat1', 'cat2', 'cat3', 'brand', 'customer_id']):
        series = data[['cat1', 'cat2', 'cat3', 'brand', 'customer_id', 'tn_scaled','suma_total_2019_scaled','suma_total_2018_scaled','variacion_feb2018_vs_dic2017','variacion_feb2019_vs_dic2018']].values
        if len(series) > 2:  # Asegurarse de que haya suficientes datos
            X.append(series[:-2])  # Todos los datos excepto los últimos 2
            y.append(series[-1, 0])  # Selecciona solo tn_scaled como objetivo

    # Pad y reshape de X
    max_len = max(len(seq) for seq in X)
    X_padded = np.array([np.pad(seq, ((max_len - len(seq), 0), (0, 0)), mode='constant') for seq in X]).astype(np.float32)
    y = np.array(y).astype(np.float32)
    
    if len(X_padded) == 0 or len(y) == 0:
        continue

    # Construir y entrenar el modelo
    model = build_model((X_padded.shape[1], X_padded.shape[2]))
    model.fit(X_padded, y, epochs=200, verbose=2, batch_size=128, validation_split=0.2, callbacks=[reduce_lr])
    models[cluster] = model

    # Hacer predicciones
    for key, data in cluster_data.groupby(['cat1', 'cat2', 'cat3', 'brand', 'customer_id']):
        series = data[['cat1', 'cat2', 'cat3', 'brand', 'customer_id', 'tn_scaled','suma_total_2019_scaled','suma_total_2018_scaled','variacion_feb2018_vs_dic2017','variacion_feb2019_vs_dic2018']].values
        
        if len(series) > 2:
            max_len = len(series) - 1
            X_pred = np.pad(series[1:], ((max_len - len(series[1:]), 0), (0, 0)), mode='constant').astype(np.float32)
            X_pred = np.reshape(X_pred, (1, X_pred.shape[0], X_pred.shape[1]))
            pred = model.predict(X_pred, verbose=0)
            predictions.append([key[0], key[1], key[2], key[3], pred[0][0], key[4]])

    # Guardar las predicciones en un DataFrame y exportar a CSV
    pred_dff_temp = pd.DataFrame(predictions, columns=['cat1', 'cat2', 'cat3', 'brand', 'prediccion', 'customer_id'])
    pred_dff_temp.to_csv(f"predicciones_temporales_cluster_{cluster}.csv", index=False)


Entrenando cluster numero: 0
Epoch 1/100
10/10 - 9s - 889ms/step - loss: 7.1365 - val_loss: 6.7846 - learning_rate: 1.0000e-04
Epoch 2/100
10/10 - 3s - 323ms/step - loss: 6.8962 - val_loss: 7.0124 - learning_rate: 1.0000e-04
Epoch 3/100
10/10 - 3s - 303ms/step - loss: 6.7143 - val_loss: 6.5734 - learning_rate: 1.0000e-04
Epoch 4/100
10/10 - 3s - 303ms/step - loss: 6.5323 - val_loss: 6.2919 - learning_rate: 1.0000e-04
Epoch 5/100
10/10 - 3s - 300ms/step - loss: 6.3984 - val_loss: 6.3032 - learning_rate: 1.0000e-04
Epoch 6/100
10/10 - 3s - 302ms/step - loss: 6.1968 - val_loss: 6.2002 - learning_rate: 1.0000e-04
Epoch 7/100
10/10 - 3s - 304ms/step - loss: 6.0634 - val_loss: 5.9017 - learning_rate: 1.0000e-04
Epoch 8/100
10/10 - 3s - 293ms/step - loss: 5.8915 - val_loss: 5.8784 - learning_rate: 1.0000e-04
Epoch 9/100
10/10 - 3s - 304ms/step - loss: 5.7615 - val_loss: 5.5661 - learning_rate: 1.0000e-04
Epoch 10/100
10/10 - 3s - 287ms/step - loss: 5.6111 - val_loss: 5.7368 - learning_rate: 1

In [58]:
pred_dff_temp.head()

,cat1,cat2,cat3,brand,prediccion,customer_id
0,0,0,4,22,1.002434,10001
1,0,0,4,22,1.002435,10032
2,0,0,4,22,1.002435,10106
3,0,0,4,22,1.002435,10144
4,0,0,4,22,1.002436,10225


In [59]:
pred_dff_temp.to_csv("C:/Users/Usuario/desktop/vero2/pred_df_temp.csv", index=False)


In [60]:
pred_dff = pd.DataFrame(predictions, columns=['cat1', 'cat2', 'cat3', 'brand', 'prediccion', 'customer_id'])
# Suponiendo que `scaler` es el objeto StandardScaler que usaste para escalar `tn` originalmente

# Desescalar las predicciones
predicciones_desescaladas = scaler.inverse_transform(pred_dff['prediccion'].values.reshape(-1, 1))

# Reemplazar las predicciones escaladas con las desescaladas en el DataFrame
pred_dff['prediccion'] = predicciones_desescaladas.flatten()

# Ahora pred_df tiene las predicciones desescaladas en la columna 'prediccion'
pred_dff.head()


,cat1,cat2,cat3,brand,prediccion,customer_id
0,0,0,4,22,7.409166,10001
1,0,0,4,22,7.409168,10032
2,0,0,4,22,7.409173,10106
3,0,0,4,22,7.409173,10144
4,0,0,4,22,7.409177,10225


In [64]:
# Paso 1: Obtener combinaciones únicas de 'product_id', 'cat1', 'cat2', 'cat3', 'brand' del DataFrame original
distinct_combinations = df[['product_id', 'cat1', 'cat2', 'cat3', 'brand']].drop_duplicates()

# Paso 2: Realizar un join con pred_df en las columnas correspondientes
pred_dff = pred_dff.merge(distinct_combinations, on=['cat1', 'cat2', 'cat3', 'brand'], how='left')

# Mostrar el DataFrame con las predicciones y las nuevas columnas agregadas
print(pred_dff.head())


   cat1  cat2  cat3  brand  prediccion  customer_id  product_id
0     0     0     4     22    7.409166        10001       20609
1     0     0     4     22    7.409168        10032       20609
2     0     0     4     22    7.409173        10106       20609
3     0     0     4     22    7.409173        10144       20609
4     0     0     4     22    7.409177        10225       20609


In [65]:
# Inicializar una lista para almacenar las predicciones ajustadas
predictions_adjusted = []

# Iterar sobre las filas de pred_df
for index, row in pred_dff.iterrows():
    key = (row['cat1'], row['cat2'], row['cat3'], row['brand'], row['product_id'])
    
    # Buscar el ratio correspondiente en ratio_dict utilizando la clave correcta
    if key in ratio_dict:
        ratio = ratio_dict[key]
        # Calcular la predicción ajustada
        prediccion_ajustada = row['prediccion'] * ratio
        predictions_adjusted.append(prediccion_ajustada)
    else:
        predictions_adjusted.append(row['prediccion'])  # Mantener la predicción original si no hay ratio definido

# Agregar las predicciones ajustadas al DataFrame pred_df
pred_dff['prediccion_ajustada'] = predictions_adjusted

# Mostrar el DataFrame con las predicciones ajustadas
print(pred_dff.head())


   cat1  cat2  cat3  brand  prediccion  customer_id  product_id  \
0     0     0     4     22    7.409166        10001       20609   
1     0     0     4     22    7.409168        10032       20609   
2     0     0     4     22    7.409173        10106       20609   
3     0     0     4     22    7.409173        10144       20609   
4     0     0     4     22    7.409177        10225       20609   

   prediccion_ajustada  
0             7.409166  
1             7.409168  
2             7.409173  
3             7.409173  
4             7.409177  


In [66]:
# Sumarizar las predicciones ajustadas por product_id
summarized_predictions = pred_dff.groupby('product_id')['prediccion_ajustada'].sum().reset_index()

# Mostrar el DataFrame con las predicciones sumarizadas
print(summarized_predictions)


     product_id  prediccion_ajustada
0         20001          2499.465930
1         20002          2510.942963
2         20003          3151.594747
3         20004          3151.594747
4         20005          3151.594747
..          ...                  ...
775       21263           449.055525
776       21265           892.007104
777       21266           892.007104
778       21267           179.697158
779       21276           179.697158

[780 rows x 2 columns]


In [67]:
# Guardar las predicciones finales en un archivo CSV
summarized_predictions.to_csv("C:/Users/Usuario/desktop/vero2/modelodtw2002.csv", index=False)